In [1]:
!pip install langchain chromadb tiktoken transformers sentence_transformers pypdf
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [3]:
#from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split()

In [4]:
# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=tiktoken_len
)

docs = text_splitter.split_documents(pages)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}
hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
db = Chroma.from_documents(docs, hf)

In [7]:
# query it
query = '장기저탄소발전전략'
docs = db.similarity_search(query)

print(docs[0].page_content)

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


In [8]:
# save to disk
db2 = Chroma.from_documents(docs, hf , persist_directory='./chroma_db')
docs = db2.similarity_search(query)

In [9]:
db3 = Chroma(persist_directory='./chroma_db', embedding_function=hf)
docs= db3.similarity_search(query)

print(docs[0].page_content)

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


In [10]:
docs = db3.similarity_search_with_relevance_scores(query, k=3)

print('가장 유사한 문서: \n\n {}\n\n'.format(docs[0][0].page_content))
print('문서 유사도:\n {}'.format(docs[0][1]))

가장 유사한 문서: 

 05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


문서 유사도:
 0.5288704602664074


# FAISS

In [11]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.9 MB/s eta 0:00:00


In [12]:
from langchain.vectorstores import FAISS

pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=tiktoken_len
)
docs = text_splitter.split_documents(pages)

model_name='jhgan/ko-sbert-nli'
model_kwargs={'device':'cpu'}
encode_kwargs={'normalize_embeddings':True}
ko=HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [13]:
db = FAISS.from_documents(docs, ko)

In [14]:
query = '삼성전자의 동향'
docs = db.similarity_search(query)
print(docs[0].page_content)


07이노비즈 정책브리프
04 국내 대기업 동향 3)
∙SK그룹
- ‘20년 11월 국내 최초로 RE100 프로젝트에 가입
- 2025년까지 이천·청주 사업장의 1톤 차량을 전기 자동차로 교체할 예정
- SK하이닉스는 공정가스 제거시스템을 적용하는 사업장을 확대함
∙ 삼성전자
- 2019년 온실가스 배출량을 ‘18년 대비 8.9%, ‘08년 대비 70% 감축
- 수원사업장과 기흥사업장에 각각 1.9매가와트 , 1.5메가와트 규모의 태양광발전 설비 설치
- 제품 사용단계에서의 고효율 제품 및 저전력 반도체 개발을 통해 전력 효율화 추진
∙ LG그룹
- 15년째 세계 최대 지구촌 전등 끄기 캠페인인 ‘어스아워 ’에 동참
- 이사회 산하에 ESG 위원회 신설
- LG전자는 2030년까지 탄소중립을 달성하겠다는 목표를 수립하고 , 포스코와 협력하여 해수열냉난방시스
템, 건물일체형태양광 등 신재생에너지 관련 기술 개발에 적극 투자
∙ 현대제철


In [15]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

[(Document(page_content='07이노비즈 정책브리프\n04 국내 대기업 동향 3)\n∙SK그룹\n- ‘20년 11월 국내 최초로 RE100 프로젝트에 가입\n- 2025년까지 이천·청주 사업장의 1톤 차량을 전기 자동차로 교체할 예정\n- SK하이닉스는 공정가스 제거시스템을 적용하는 사업장을 확대함\n∙ 삼성전자\n- 2019년 온실가스 배출량을 ‘18년 대비 8.9%, ‘08년 대비 70% 감축\n- 수원사업장과 기흥사업장에 각각 1.9매가와트 , 1.5메가와트 규모의 태양광발전 설비 설치\n- 제품 사용단계에서의 고효율 제품 및 저전력 반도체 개발을 통해 전력 효율화 추진\n∙ LG그룹\n- 15년째 세계 최대 지구촌 전등 끄기 캠페인인 ‘어스아워 ’에 동참\n- 이사회 산하에 ESG 위원회 신설\n- LG전자는 2030년까지 탄소중립을 달성하겠다는 목표를 수립하고 , 포스코와 협력하여 해수열냉난방시스\n템, 건물일체형태양광 등 신재생에너지 관련 기술 개발에 적극 투자\n∙ 현대제철', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 9}),
  0.96180105),
 (Document(page_content='이노비즈 정책브리프\n발행인 |임병훈\n편집인 |김세종\n발행일 |2021년 5월 18일\n발행처 |이노비즈정책연구원\n경기도 성남시 분당구 판교로 255(삼평동 ) 이노밸리 E동 202호\n전화 : 031-628-9600   팩스 : 031-628-9611   홈페이지 : http://www.innobiz.or.kr\n※ 본지에 게재된 내용은 필자 개인(연구진 )의 견해이며 , 이노비즈협회의 공식 견해와 일치하지 않을 수도 있습니다 .\n※ 본지의 내용은 상업적으로 사용할 수 없으며 , 내용을 인용할 때는 반드시 출처를 밝혀주시기 바랍니다 .\n※ 본지는 일반 학술연구와는 달리 시의성 , 정책현안의 긴급성 등의 이유로 포괄적 ·개괄적

In [16]:
db.save_local('faiss_index')

In [21]:
new_db = FAISS.load_local('faiss_index', ko, allow_dangerous_deserialization=True)

query = '탄소중립 사회'
docs = new_db.similarity_search_with_relevance_scores(query, k=3)

In [22]:
print('질문: {} \n'.format(query))
for i in range(len(docs)):
  print('{0}번째 유사 문서 유사도 \n{1}'.format(i+1, round(docs[i][1],2)))
  print('-' * 100)
  print(docs[i][0].page_content)
  print('\n')
  print(docs[i][0].metadata)
  print('-' * 100)

질문: 탄소중립 사회 

1번째 유사 문서 유사도 
0.66
----------------------------------------------------------------------------------------------------
- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제
공 확대 등 순환 사회로의 전환 가속화
*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 
총량을 의미함
 ③ 탄소중립 사회로의 공정전환
- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대
체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지원
- 구조전환으로 축소되는 석탄발전 ‧내연기관차 산업 등의 경우, 대체‧유망분야로의 사업전환을 적극 지원하
고 직업훈련 ·재취업 지원 등 고용친화적 전환을 위한 정책 추진방향 마련
- (지역중심의 탄소중립 실현) 지역 중심 탄소중립 실행 지원, 지역별 맞춤형 전략 이행을 위한 전략 수립


{'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 6}
----------------------------------------------------------------------------------------------------
2번째 유사 문서 유사도 
0.65
----------------------------------------------------------------------------------------------------
01이노비즈 정책브리프
01 탄소중립과 대응 필요성
탄소중립이란 ?
∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제
로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zer

In [27]:
query = '경제구조의 저탄소화'
docs = new_db.max_marginal_relevance_search(query, k=3, fetck_k=10, lamda_mult=0.3)  # 다양성을 추구한다.
len(docs)

3

In [28]:
print('질문: {} \n'.format(query))
for i in range(len(docs)):
  print('{0}번째 유사 문서 유사도'.format(i+1))
  print('-' * 100)
  print(docs[i].page_content)
  print('\n')
  print(docs[i].metadata)
  print('-' * 100)

질문: 경제구조의 저탄소화 

1번째 유사 문서 유사도
----------------------------------------------------------------------------------------------------
- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제
공 확대 등 순환 사회로의 전환 가속화
*탄소발자국이란 개인 또는 기업, 국가 등의 활동이나 상품을 생산하고 소비하는 전체 과정을 통해 발생시키는 온실가스 , 특히 이산화탄소의 
총량을 의미함
 ③ 탄소중립 사회로의 공정전환
- (취약 산업·계층 보호) 내연기관차 완성차 및 부품업체 등 축소산업에 대한 R&D, M&A 등을 통해 대
체·유망분야로 사업전환 적극 지원, 맞춤형 재취업 지원
- 구조전환으로 축소되는 석탄발전 ‧내연기관차 산업 등의 경우, 대체‧유망분야로의 사업전환을 적극 지원하
고 직업훈련 ·재취업 지원 등 고용친화적 전환을 위한 정책 추진방향 마련
- (지역중심의 탄소중립 실현) 지역 중심 탄소중립 실행 지원, 지역별 맞춤형 전략 이행을 위한 전략 수립


{'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 6}
----------------------------------------------------------------------------------------------------
2번째 유사 문서 유사도
----------------------------------------------------------------------------------------------------
03이노비즈 정책브리프
02 탄소중립 대응을 위한 정부 정책 1)
탄소중립 2050 추진전략
∙우리나라 정부는 지난 2020년 12월 7일 관계부처 합동으로 ‘2050 탄소중립 추진전략 ’을 발표함
- 정부는 탄소중립이 어렵지만 가야할 길이